Here we'll explore how we can explore the interactions between the fly and the ball precisely

In [1]:
from utils_behavior import Sleap_utils

In [5]:
# Get some fly

Fly1 = Sleap_utils.Sleap_Tracks("/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/231115_TNT_Fine_2_Videos_Tracked/arena9/corridor6/corridor6_preprocessed_full_body.h5", object_type="fly")

Ball1= Sleap_utils.Sleap_Tracks("/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/231115_TNT_Fine_2_Videos_Tracked/arena9/corridor6/corridor6_tracked_ball_preprocessed.000_corridor6_preprocessed.analysis.h5", object_type="ball")

Processing fly 1/1
Loaded SLEAP tracking file: /mnt/upramdya_data/MD/MultiMazeRecorder/Videos/231115_TNT_Fine_2_Videos_Tracked/arena9/corridor6/corridor6_preprocessed_full_body.h5
N° of objects: 1
Nodes: ['Head', 'Thorax', 'Abdomen', 'Rfront', 'Lfront', 'Rmid', 'Lmid', 'Rhind', 'Lhind', 'Rwing', 'Lwing']
Video FPS: 29.0
Processing ball 1/1
Loaded SLEAP tracking file: /mnt/upramdya_data/MD/MultiMazeRecorder/Videos/231115_TNT_Fine_2_Videos_Tracked/arena9/corridor6/corridor6_tracked_ball_preprocessed.000_corridor6_preprocessed.analysis.h5
N° of objects: 1
Nodes: ['centre']
Video FPS: 29.0


In [7]:
Dataset = Sleap_utils.CombinedSleapTracks(video_path="/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/231115_TNT_Fine_2_Videos_Tracked/arena9/corridor6/corridor6_preprocessed.mp4", sleap_tracks_list=[Fly1, Ball1])

In [8]:
Dataset.dataset

,frame,time,object,x_Head,y_Head,x_Thorax,y_Thorax,x_Abdomen,y_Abdomen,x_Rfront,y_Rfront,x_Lfront,y_Lfront,x_Rmid,y_Rmid,x_Lmid,y_Lmid,x_Rhind,y_Rhind,x_Lhind,y_Lhind,x_Rwing,y_Rwing,x_Lwing,y_Lwing,x_centre,y_centre
0,1,0.034483,fly_1,60.275352,387.490906,68.343437,395.295776,79.739174,407.505096,60.201466,375.789673,56.471191,376.235657,72.226387,371.969116,47.725174,407.876648,100.156357,395.783173,63.625931,416.905029,92.268555,419.882172,91.924477,420.078583,NaN,NaN
1,2,0.068966,fly_1,60.261478,387.412201,68.321571,395.324982,79.761124,407.548645,60.497635,372.682617,59.491940,375.991974,72.567055,372.249023,47.716373,407.946564,99.978172,395.879791,60.452927,419.569702,92.246384,419.964966,91.884872,420.172119,NaN,NaN
2,3,0.103448,fly_1,60.074608,387.992371,68.321747,395.561737,79.921944,407.650818,60.027493,375.647369,59.194965,376.072510,71.958115,372.178925,47.861851,408.094940,100.075996,395.684662,64.060440,419.908142,92.490601,419.962189,92.135849,420.165253,NaN,NaN
3,4,0.137931,fly_1,60.263126,387.408478,68.346146,395.266968,79.814934,404.611786,60.318977,375.905945,59.454235,376.160858,72.566544,371.964111,47.838470,407.902588,99.985428,395.804993,60.531090,419.525970,92.251617,419.905945,91.898682,420.105621,NaN,NaN
4,5,0.172414,fly_1,60.258152,384.276367,68.358650,395.304901,79.769409,407.538483,60.337349,375.631500,59.490604,376.049316,72.556206,372.189423,47.937557,407.839935,100.058159,395.850494,60.389820,416.672882,92.266853,419.927094,91.905304,420.127136,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208799,104398,3599.931034,ball_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,45.0
208800,104399,3599.965517,ball_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,45.0
208801,104400,3600.000000,ball_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,45.0
208802,104401,3600.034483,ball_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,45.0


In [ ]:
# Reshape the dataset to get only 1 row per frame and to have both fly and ball data in the same row
# ###&zza{#zzzaaazaedzzazazdazdz
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# }
